In [1]:
# -*- coding: utf-8 -*-
"""Untitled2.ipynb

SyntaxError: incomplete input (<ipython-input-1-25aa4d2a60f8>, line 2)

In [ ]:
Automatically generated by Colab.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1TOZuBL5GtgZh9H8K78vkVoN-ubOIfgkv
"""

In [2]:
import numpy as np
from PIL import Image
import librosa
import os

Constants (untuk lookup table permutasi)

In [3]:
INITIAL_PERMUTATION_TABLE = [58, 50, 42, 34, 26, 18, 10, 2,
                             60, 52, 44, 36, 28, 20, 12, 4,
                             62, 54, 46, 38, 30, 22, 14, 6,
                             64, 56, 48, 40, 32, 24, 16, 8,
                             57, 49, 41, 33, 25, 17, 9, 1,
                             59, 51, 43, 35, 27, 19, 11, 3,
                             61, 53, 45, 37, 29, 21, 13, 5,
                             63, 55, 47, 39, 31, 23, 15, 7]

In [4]:
FINAL_PERMUTATION_TABLE = [40, 8, 48, 16, 56, 24, 64, 32,
                           39, 7, 47, 15, 55, 23, 63, 31,
                           38, 6, 46, 14, 54, 22, 62, 30,
                           37, 5, 45, 13, 53, 21, 61, 29,
                           36, 4, 44, 12, 52, 20, 60, 28,
                           35, 3, 43, 11, 51, 19, 59, 27,
                           34, 2, 42, 10, 50, 18, 58, 26,
                           33, 1, 41, 9, 49, 17, 57, 25]

PC1 and PC2 for key schedule lookup

In [5]:
PC1 = [57, 49, 41, 33, 25, 17, 9,
       1, 58, 50, 42, 34, 26, 18,
       10, 2, 59, 51, 43, 35, 27,
       19, 11, 3, 60, 52, 44, 36,
       63, 55, 47, 39, 31, 23, 15,
       7, 62, 54, 46, 38, 30, 22,
       14, 6, 61, 53, 45, 37, 29,
       21, 13, 5, 28, 20, 12, 4]

In [6]:
PC2 = [14, 17, 11, 24, 1, 5, 3, 28,
       15, 6, 21, 10, 23, 19, 12, 4,
       26, 8, 16, 7, 27, 20, 13, 2,
       41, 52, 31, 37, 47, 55, 30, 40,
       51, 45, 33, 48, 44, 49, 39, 56,
       34, 53, 46, 42, 50, 36, 29, 32]

In [7]:
SHIFT_SCHEDULE = [1, 1, 2, 2, 2, 2, 2, 2,
                  1, 2, 2, 2, 2, 2, 2, 1]

Expansion Table (32-bit to 48-bit)

In [8]:
EXPANSION_TABLE = [32, 1, 2, 3, 4, 5,
                   4, 5, 6, 7, 8, 9,
                   8, 9, 10, 11, 12, 13,
                   12, 13, 14, 15, 16, 17,
                   16, 17, 18, 19, 20, 21,
                   20, 21, 22, 23, 24, 25,
                   24, 25, 26, 27, 28, 29,
                   28, 29, 30, 31, 32, 1]

P-Box Permutation Table

In [9]:
P_BOX = [16, 7, 20, 21,
         29, 12, 28, 17,
         1, 15, 23, 26,
         5, 18, 31, 10,
         2, 8, 24, 14,
         32, 27, 3, 9,
         19, 13, 30, 6,
         22, 11, 4, 25]

In [10]:
S_BOXES = [
# S1
[
[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
[0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
[4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
[15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
],
# S2
[
[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
[3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
[0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
[13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
],
# S3
[
[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
[13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
[13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
[1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
],
# S4
[
[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
[13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
[10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
[3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
],
# S5
[
[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
[14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
[4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
[11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
],
# S6
[
[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
[10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
[9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
[4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
],
# S7
[
[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
[13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
[1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
[6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
],
# S8
  [
  [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
  [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
  [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
  [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
  ]
]

Standard DES permutation table P

In [11]:
P = [
    16, 7, 20, 21,
    29, 12, 28, 17,
    1, 15, 23, 26,
    5, 18, 31, 10,
    2, 8, 24, 14,
    32, 27, 3, 9,
    19, 13, 30, 6,
    22, 11, 4, 25
]

In [12]:
def generate_des_key(key_number):
    binary_str = format(key_number, '064b')
    return np.array([int(bit) for bit in binary_str], dtype=np.uint8)

In [13]:
def pad_data(data):
    pad_length = 8 - (len(data) % 8)
    return np.pad(data, (0, pad_length), 'constant', constant_values=(pad_length,))

In [14]:
def unpad_data(data):
    pad_length = data[-1]
    return data[:-pad_length]

In [15]:
def generate_des_key(key_number):
    binary_str = format(key_number, '064b')
    return np.array([int(bit) for bit in binary_str], dtype=np.uint8)

In [16]:
def permute(block, table):
    return np.array([block[i - 1] for i in table])

In [17]:
def initial_permutation(block):
    return permute(block, INITIAL_PERMUTATION_TABLE)

In [18]:
def final_permutation(block):
    return permute(block, FINAL_PERMUTATION_TABLE)

In [19]:
def key_schedule(key):
    # Implementasi key schedule yang sebenarnya
    # Ini adalah penyederhanaan, Anda perlu mengimplementasikan algoritma key schedule DES yang sebenarnya
    subkeys = []
    for i in range(16):
        subkey = np.roll(key, -i)[:48]  # Ini hanya contoh, bukan implementasi DES yang benar
        subkeys.append(subkey)
    return subkeys

In [20]:
def substitute(expanded_block):
    substituted = np.zeros(32, dtype=np.uint8)
    for i in range(8):
        block = expanded_block[i*6:(i+1)*6]
        row = int(block[0]) << 1 | int(block[5])
        col = (int(block[1]) << 3) | (int(block[2]) << 2) | (int(block[3]) << 1) | int(block[4])
        val = S_BOXES[i][row][col]
        substituted[i*4:(i+1)*4] = [(val >> j) & 1 for j in range(3, -1, -1)]
    return substituted

In [21]:
def f_function(right, subkeys):
    # Perform substitution and permutation using P
    substituted = substitute_function(right)  # Assuming you have a substitute function
    return permute(substituted, P)  # Use the P permutation table
# DES S-boxes (S1 to S8)
S_BOX = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
    ],
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
    ],
    # Add S3 to S8 here...
]

In [28]:
def substitute_function(right):
    substituted_output = []

    # Pastikan panjang input ke substitusi adalah 48 bit
    assert len(right) == 48, "Input length for substitution must be 48 bits."

    # Substitusi menggunakan 8 S-Boxes
    for i in range(8):
        block = right[i * 6: (i + 1) * 6]  # Ambil 6 bit per blok
        assert len(block) == 6, f"Block {i} is not 6 bits long."

        row = int(f"{block[0]}{block[5]}", 2)  # Ambil baris dari bit pertama dan terakhir
        col = int(''.join([str(b) for b in block[1:5]]), 2)  # Ambil kolom dari 4 bit tengah

        # Gunakan S-Box yang sesuai, periksa jika row dan col valid
        try:
            s_box_value = S_BOXES[i][row][col]
        except IndexError as e:
            print(f"IndexError at S-Box {i}: row={row}, col={col}")
            raise e

        # Ubah nilai 4-bit menjadi biner
        substituted_output.append(format(s_box_value, '04b'))

    # Gabungkan semua output substitusi menjadi string biner 32-bit
    return ''.join(substituted_output)

def f_function(right, subkey):
    # Ekspansi 32-bit menjadi 48-bit menggunakan tabel ekspansi
    expanded = permute(right, [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17, 16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1])

    # XOR hasil ekspansi dengan subkey
    assert len(expanded) == 48, "Expanded block must be 48 bits."
    xor_result = np.bitwise_xor(expanded, subkey)

    # Substitusi hasil XOR melalui S-Boxes
    substituted = substitute_function(xor_result)

    # Hasil substitusi diubah menjadi P-Box permutasi (32-bit)
    return permute([int(bit) for bit in substituted], P_BOX)


In [23]:
def des_encrypt_block(block, subkeys):
    block = initial_permutation(block)
    left, right = block[:32], block[32:]
    for i in range(16):
        left, right = right, np.bitwise_xor(left, f_function(right, subkeys[i]))
    block = np.concatenate((right, left))
    return final_permutation(block)

In [24]:
def des_decrypt_block(block, subkeys):
    block = initial_permutation(block)
    left, right = block[:32], block[32:]
    for i in range(15, -1, -1):
        left, right = right, np.bitwise_xor(left, f_function(right, subkeys[i]))
    block = np.concatenate((right, left))
    return final_permutation(block)

Encryption/Decryption for Images

In [25]:
def process_image(file_path, key, output_path, mode='encrypt'):
    print(f"Processing image: {file_path}")

    # Baca gambar
    image = Image.open(file_path)
    image_data = np.array(image)

    # Generate subkeys
    subkeys = key_schedule(key)

    # Proses setiap channel
    processed_channels = []
    print(f"Original Size : {len(image_data)}")
    print(image_data.shape)
    for channel in range(image_data.shape[2]):
        channel_data = image_data[:,:,channel].flatten()

        # Konversi ke bits
        bits = np.unpackbits(channel_data)
        print(f"Total Size Before Padding : {len(bits)}")

        # Pad data jika diperlukan
        if len(bits) % 64 != 0:
            pad_length = 64 - (len(bits) % 64)
            bits = np.pad(bits, (0, pad_length), 'constant', constant_values=0)
        total_blocks = len(bits)
        print(f"Total Size After Padding : {total_blocks}")

        # Proses setiap blok 64-bit
        processed_bits = []
        for i in range(0, len(bits), 64):
            block = bits[i:i+64]
            print(f"Start procesing Block {i} from {total_blocks} ({i * 100 / (total_blocks*3)} %)")
            if mode == 'encrypt':
                processed_block = des_encrypt_block(block, subkeys)
            else:
                processed_block = des_decrypt_block(block, subkeys)
            processed_bits.extend(processed_block)

        # # Konversi kembali ke bytes
        processed_channel = np.packbits(np.array(processed_bits))

        # # Reshape kembali ke bentuk channel asli
        processed_channels.append(processed_channel[:image_data.shape[0]*image_data.shape[1]].reshape(image_data.shape[:2]))

    # Gabungkan channel yang telah diproses
    processed_image = np.stack(processed_channels, axis=-1)

    # # Simpan gambar hasil
    Image.fromarray(processed_image.astype(np.uint8)).save(output_path)
    print(f"Finished processing image: {output_path}")
    return output_path

Usage example

In [33]:
if __name__ == "__main__":
    # Key derived from a number
    key_number = 11422081
    key = generate_des_key(key_number)

    # Image encryption
    print("Starting image encryption")
    encrypted_image = process_image("penn.jpg", key, "penn_encrypted.jpg", mode='encrypt')
    print(f"Image encrypted: {encrypted_image}")

    # Image decryption
    # print("Starting image decryption")
    decrypted_image = process_image("penn.jpg", key, "penn_decrypted.jpg", mode='decrypt')
    print(f"Image decrypted: {decrypted_image}")

Output streaming akan dipotong hingga 5000 baris terakhir.
Start procesing Block 9510528 from 9830400 (32.248697916666664 %)
Start procesing Block 9510592 from 9830400 (32.24891493055556 %)
Start procesing Block 9510656 from 9830400 (32.24913194444444 %)
Start procesing Block 9510720 from 9830400 (32.249348958333336 %)
Start procesing Block 9510784 from 9830400 (32.24956597222222 %)
Start procesing Block 9510848 from 9830400 (32.249782986111114 %)
Start procesing Block 9510912 from 9830400 (32.25 %)
Start procesing Block 9510976 from 9830400 (32.250217013888886 %)
Start procesing Block 9511040 from 9830400 (32.25043402777778 %)
Start procesing Block 9511104 from 9830400 (32.250651041666664 %)
Start procesing Block 9511168 from 9830400 (32.25086805555556 %)
Start procesing Block 9511232 from 9830400 (32.25108506944444 %)
Start procesing Block 9511296 from 9830400 (32.251302083333336 %)
Start procesing Block 9511360 from 9830400 (32.25151909722222 %)
Start procesing Block 9511424 from 98